In [17]:
import re
import pandas as pd
import unicodedata

# Définit les variantes de “de”
PREPOSITIONS = [
    r"\bde la\b",
    r"\bde\b",
    r"\bdu\b",
    r"\bdes\b",
    r"\bd’\b",
    r"\bd’un\b",
    r"\bd’une\b",
    r"\bde l’\b"
]

pattern = re.compile(
    rf"(.+?)\s+(?:{'|'.join(PREPOSITIONS)})\s+(.+?)(?:\s*\|\s*(.*))?$",
    flags=re.IGNORECASE
)

def nettoyer_texte(s: str) -> str:
    """Nettoyage de base + amélioré : trim, minuscules, retire ponctuation, accents, espaces multiples."""
    if s is None:
        return None
    s = s.strip()
    s = s.lower()
    # Normaliser accents → forme de base (optionnel)
    s = unicodedata.normalize('NFKD', s)
    s = ''.join([c for c in s if not unicodedata.combining(c)])
    # Retirer ponctuation interne (on considère que mot1/mot2 sont des mots simples)
    s = re.sub(r"[^\w\s]", " ", s)  # remplace tout ce qui n’est pas lettre/chiffre/_/espace
    # Remplacer chiffres si non pertinents (ex ici on retire les chiffres)
    s = re.sub(r"\d+", "", s)
    # Remplacer plusieurs espaces par un seul
    s = re.sub(r"\s+", " ", s)
    s = s.strip()
    return s

def extraire_paires(ligne: str):
    m = pattern.match(ligne)
    if not m:
        return None
    mot1_raw, mot2_raw, relation_raw = m.groups()
    mot1 = nettoyer_texte(mot1_raw)
    mot2 = nettoyer_texte(mot2_raw)
    relation = nettoyer_texte(relation_raw) if relation_raw else "r_has_causatif"
    # On peut rejeter les cas où mot1 ou mot2 après nettoyage sont vides
    if not mot1 or not mot2:
        return None
    return mot1, mot2, relation

def traiter_fichier(input_path: str, output_csv: str):
    résultats = []
    with open(input_path, encoding="utf‑8") as f:
        for lineno, ligne in enumerate(f, start=1):
            ligne = ligne.strip()
            if not ligne:
                continue
            paire = extraire_paires(ligne)
            if paire:
                mot1, mot2, relation = paire
                résultats.append({"mot1": mot1, "mot2": mot2, "relation": relation})
            else:
                # Optionnel : logger ou compter
                # print(f"Ligne {lineno} non extraite : {ligne}")
                pass

    df = pd.DataFrame(résultats)
    # Optionnel : drop duplicates
    df = df.drop_duplicates(subset=["mot1", "mot2", "relation"])
    # Optionnel : reset index
    df = df.reset_index(drop=True)
    df.to_csv(output_csv, index=False, encoding="utf‑8")
    print(f"{len(résultats)} paires extraites, écrites dans {output_csv}")

if __name__ == "__main__":
    traiter_fichier("/content/sample_data/liste_paires.txt", "paires_nettoyees.csv")


16 paires extraites, écrites dans paires_nettoyees.csv
